In [1]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
%matplotlib inline

from entities import CoachingPracticeFinance
from presentation import DataFrameRenderer

practice = CoachingPracticeFinance.load()

In [2]:
aggregated = [
    [c.code, c.name, c.contract, c.contact_name, c.contact_phone, c.contact_email] for c in practice.consultancies
]
df = pd.DataFrame(aggregated, columns=["Code", "Name", "Contract", "Contact Name", "Contact Phone", "Contact Email"])
output = DataFrameRenderer(title="Consultancies", data=df, sort_by=["Name"])
print(output)


Consultancies

Code    Name                           Contract     Contact Name    Contact Phone    Contact Email
ilc     Industrial Logic Canada        CTR018457    Ardita Karaj    416-123-4567     ardita@industriallogic.ca
qdi     Quality Driven Inc.            Stand-Alone  Paul Carvalho   416-123-4567     paul@quality-driven.com
ra      Rosetta Technology Group, LLC  CTR019969    Jardena London  999-555-1212     jardena@rosettaagile.com
xodiac  Xodiac (2018) Inc.             CTR020388    Gino Marckx     416-123-4567     gino.marckx@xodiac.ca


In [3]:
areas = []
for o in practice.contractors:
    for a in practice.support_areas:
        for s in practice.area_assignments:
            if o.code == s.contractor_code and s.support_area_code == a.code:
                areas.append([a.name, o.name, s.funding_source_transit, s.start_date, s.end_date])
df = pd.DataFrame(areas, columns=["Area", "Contractor", "Funding Source", "Start Date", "End Date"])
output = DataFrameRenderer(title="Area Assignments", data=df, sort_by=["Area", "Contractor"])
print(output)


Area Assignments

Area                Contractor      Funding Source  Start Date    End Date
Commercial Banking  John Borys                5332  2023-05-01    2023-10-31


In [4]:
invoices = []
for o in practice.consultancies:
    for i in o.invoices:
        invoices.append([i.number, i.issue_date.isoformat(), i.total(), o.name, i.paid])
df = pd.DataFrame(invoices, columns=["Invoice Number", "Issue Date", "Total", "Consultancy", "Paid"])
output = DataFrameRenderer(title="Invoices", data=df, sort_by=["Issue Date", "Invoice Number"])
print(output)


Invoices

  Invoice Number  Issue Date    Total       Consultancy              Paid
           12345  2023-06-27    $45,987.00  Industrial Logic Canada  True


In [5]:
people = []
for o in practice.consultancies:
    for p in practice.contractors:
        if o.code == p.consultancy_code:
            people.append([p.code, p.name, o.name])
for p in practice.employees:
    people.append([p.code, p.name, "RBC"])
df = pd.DataFrame(people, columns=["Code", "Name", "Company"])
output = DataFrameRenderer(title="People", data=df, sort_by=["Name"])
print(output)


People

Code       Name                Company
267298933  Adeel Ali           Industrial Logic Canada
267554517  Ali Jafri           RBC
ajafri     Ali Jafri           Rosetta Technology Group, LLC
819021528  Carlos Puche        RBC
cliriano   Cristina Liriano    Rosetta Technology Group, LLC
271195968  Ellen Grove         Industrial Logic Canada
339011652  Gino Marckx         Xodiac (2018) Inc.
308959444  Gord Mak            RBC
332052554  Jardena London      Rosetta Technology Group, LLC
271055642  Jason Sallis        RBC
267298982  John Borys          Industrial Logic Canada
325710770  Kartik Nagarajan    RBC
267958346  Lia Baciu           RBC
267298834  Matt Barcomb        Industrial Logic Canada
265846691  Mike Bowler         Industrial Logic Canada
264823469  Paul Carvalho       Quality Driven Inc.
347793614  Peter Wong          RBC
267747404  Ram Bathija         Rosetta Technology Group, LLC
348438151  Robert French       RBC
338191463  Robert MacGregor    Industrial Logic Cana

In [6]:
from datetime import date

contractors = []
for consultancy in practice.consultancies:
    for o in practice.contractors:
        if o.consultancy_code == consultancy.code:
            ta_found = False
            for ta in practice.transaction_agreements:
                if ta.contractor_code == o.code:
                    if ta.start_date <= date.today() <= ta.end_date:
                        ta_found = True
                        contractors.append(
                            [o.code, o.name, consultancy.name, ta.number, ta.rate, ta.hours, ta.start_date, ta.end_date])
            if not ta_found:
                contractors.append([o.code, o.name, consultancy.name, None, None, "", None, None])
df = pd.DataFrame(contractors, columns=["Code", "Name", "Consultancy", "Agreement", "Rate", "Hours", "Start", "End"])
output = DataFrameRenderer(title="Contractors", data=df, sort_by=["Name"])
print(output)


Contractors

Code       Name                Consultancy                    Agreement    Rate     Hours    Start       End
267298933  Adeel Ali           Industrial Logic Canada        ILC-2021-02  $275.00  100      2023-05-01  2023-10-31
ajafri     Ali Jafri           Rosetta Technology Group, LLC
cliriano   Cristina Liriano    Rosetta Technology Group, LLC
271195968  Ellen Grove         Industrial Logic Canada
339011652  Gino Marckx         Xodiac (2018) Inc.
332052554  Jardena London      Rosetta Technology Group, LLC
267298982  John Borys          Industrial Logic Canada        ILC-2021-01  $275.00  100      2023-05-01  2023-10-31
267298834  Matt Barcomb        Industrial Logic Canada
265846691  Mike Bowler         Industrial Logic Canada        ILC-2021-02  $275.00  100      2023-04-01  2023-09-16
264823469  Paul Carvalho       Quality Driven Inc.
267747404  Ram Bathija         Rosetta Technology Group, LLC
338191463  Robert MacGregor    Industrial Logic Canada
265072959  Shawn Bu

In [7]:
consultancies = []
for o in practice.consultancies:
    for c in practice.contractors:
        if o.code == c.consultancy_code:
            consultancies.append([o.code, o.name, c.code])
df = pd.DataFrame(consultancies, columns=["Code", "Name", "Contractors"])
aggregated = df.groupby(["Code", "Name"], as_index=False).count()
output = DataFrameRenderer(title="Consultancies", data=aggregated, sort_by=["Name"])
print(output)


Consultancies

Code    Name                             Contractors
ilc     Industrial Logic Canada                    8
qdi     Quality Driven Inc.                        1
ra      Rosetta Technology Group, LLC              4
xodiac  Xodiac (2018) Inc.                         2


In [8]:
target_budget = []
for fs in practice.funding_sources:
    funding_range = pd.bdate_range(ta.start_date, ta.end_date, freq="C", holidays=practice.statutory_holiday_list)
    for d in funding_range:
        target_budget.append([fs.transit, fs.name, d, fs.total.__root__/len(funding_range)])
df = pd.DataFrame(target_budget, columns=["Transit", "Funding Source", "Date", "Target Budget"])
df["Month"] = df.Date.dt.month
df["Amount"] = df["Target Budget"].astype(float)
monthly_target_budget = df[["Month","Amount"]].groupby("Month", sort=False).sum()
monthly_target_budget


AttributeError: 'Money' object has no attribute '__root__'